# Softmax 함수를 이용한 분류

softmax 함수를 사용하는 이유는 다중 분류를 수행할때, 로짓 값 혹은 각각 개별에 대한 확률값을 반환받게 되면 비교하기가 어렵기 때문에, 출력값의 합을 1로 정규화 하여 적절하게 비교할 수 있도록 한다.

In [90]:
import csv
import numpy as np
import matplotlib.pyplot as plt

In [91]:
np.random.seed(1024)

RND_MEAN = 0
RND_STD = 0.003

LEARNING_RATE = 0.003

In [92]:
def init_model():
    global weight, bias, input_cnt, output_cnt
    weight = np.random.normal(RND_MEAN, RND_STD, [input_cnt, output_cnt])
    bias = np.zeros([output_cnt])

In [93]:
def multi_class_exc(epoch_count=10, mb_size=10, report=1, train_rate=0.7):
    multi_load_dataset()
    init_model()
    train_and_test(epoch_count, mb_size, report, train_rate)

In [94]:
def multi_load_dataset():
    with open("./Data/faults.csv") as csvfile:
        csvreader = csv.reader(csvfile)
        next(csvreader,None)
        rows=[]
        for row in csvreader:
            rows.append(row)

    global data, input_cnt, output_cnt

    input_cnt, output_cnt = 27,7
    data = np.asarray(rows,dtype='float32')

In [95]:
def arrange_data(mb_size, train_rate):
    global data, shuffle_map, test_begin_idx
    shuffle_map = np.arange(data.shape[0])
    np.random.shuffle(shuffle_map)

    step_count = int(data.shape[0] * train_rate) // mb_size
    test_begin_idx = step_count * mb_size
    
    return step_count

In [96]:
def get_train_data(mb_size, nth):
    global data, shuffle_map, test_begin_idx, output_cnt
    if nth == 0:
        np.random.shuffle(shuffle_map[:test_begin_idx])
    train_data = data[shuffle_map[mb_size*nth : mb_size*(nth+1)]]

    return train_data[:, :-output_cnt], train_data[:, -output_cnt:]

In [97]:
def get_test_data():
    global data, shuffle_map, test_begin_idx, output_cnt

    test_data = data[shuffle_map[test_begin_idx:]]
    return test_data[:, :-output_cnt] , test_data[:, -output_cnt:]

In [98]:
def run_train(x,y):
    output, aux_nn = forward_neuralnet(x)
    loss, aux_pp = forward_postproc(output, y)
    accuracy = eval_accuracy(output, y)

    # 역전파 수행 및 파라미터 갱신(aux_pp = [y, output, entropy])
    G_output = backprop_postproc(aux_pp)
    # aux_nn은 x.transpose()를 위해
    backprop_neuralnet(G_output, aux_nn)

    return loss, accuracy

In [99]:
def run_test(test_x, test_y):
    output, _ = forward_neuralnet(test_x)
    accuracy = eval_accuracy(output, test_y)
    
    return accuracy

In [100]:
def forward_neuralnet(x):
    '''
    신경망 연산을 수행하는 함수
    '''
    global weight, bias

    output = np.matmul(x, weight) + bias
    return output, x

In [101]:
def forward_postproc(output, y):
    '''
    Loss 값을 계산하는 함수
    '''
    entropy = softmax_cross_entropy_with_logits(y,output)
    loss = np.mean(entropy)

    # y 값을 그대로 반환하는 이유는 혹시나 값이 변하거나 다시 섞이는걸 막기 위함이다.
    return loss, [y, output, entropy]

$softmax = \frac{e^{x_i-x_k}}{e^{x_1-x_k}+e^{x_2-x_k}+\cdots+e^{x_k-x_k}+e^{x_n-x_k}}$

In [1]:
def softmax(x):
    '''
    softmax 활성화 함수
    '''
    # output 값 중 최댓값을 열 기준에서 추출하기 위하여 axis=1 설정
    max_elem = np.max(x, axis =1)

    # 바로는 계산이 안되기때문에 전치를 해서 계산 후에 다시 원상복귀 해주는 방식을 이용한다
    diff = (x.transpose() - max_elem).transpose()
    exp = np.exp(diff)
    sum_exp = np.sum(exp, axis=1)
    probs = (exp.transpose()/sum_exp).transpose()

    return probs        # softmax를 통과한 확률


def relu(x):
    return np.maximum(x,0)

In [103]:
def softmax_cross_entropy_with_logits(labels, logits):
    '''
    크로스엔트로피를 구하는 함수
    '''
    # 소프트맥스 교차 엔트로피 개선식의 신경망 예측에 대한 확률분포 q 위치에 대한 log 연산의 폭주를 막기위해
    # 아주 작은 양수값과 함ㅈ께 더해주며 위치시킨다.
    
    probs = softmax(logits)
    # 기존 확률값에 아주 작은 양수를 더해 log 함수의 오버플로우를 막음
    return -np.sum(labels * np.log(probs+1.0e-10), axis=1)

In [104]:
def backprop_postproc(aux_pp):
    y, output, entorpy = aux_pp
    G_loss = 1.0

    g_loss_entropy = 1.0 / np.prod(entorpy.shape)
    g_entropy_output = softmax_cross_entropy_with_logits_derv(y, output)

    G_entropy = g_loss_entropy * G_loss
    G_output = g_entropy_output * G_entropy

    return G_output

In [105]:
def softmax_cross_entropy_with_logits_derv(labels, logits):
    return softmax(logits) - labels

In [106]:
def backprop_neuralnet(G_output, x):
    global weight, bias
    g_output_w = x.transpose()

    G_w = np.matmul(g_output_w, G_output)
    G_b = np.sum(G_output, axis =0)

    weight -= LEARNING_RATE * G_w
    bias -= LEARNING_RATE * G_b


In [107]:
def eval_accuracy(output, y):
    estimate = np.greater(output,0)
    answer = np.greater(y,0.5)
    correct = np.equal(estimate, answer)

    return np.mean(correct)

In [108]:
def train_and_test(epoch_count, mb_size, report, train_rate):
    '''
    학습과 테스트를 하는 함수
    '''
    # 1. 데이터를 섞어주는 기능(mb_size를 이용해서 한 에폭에 필요한 미니배치 스텝수를 step_count에 저장)
    # 학습데이터와 평가 데이터를 섞어주는 기능 수행
    step_count = arrange_data(mb_size, train_rate)
    # 2. 테스트 데이터를 따로 저장하는 함수
    test_x, test_y = get_test_data()

    # 학습을 에폭 수만큼 수행
    for epoch in range(epoch_count):
        losses,accs = [], []

        # step_count를 활용해 미니배치 처리를 수행
        # step_count가 다 돌면 1에폭
        for n in range(step_count):
            train_x, train_y = get_train_data(mb_size, n)
            loss, acc = run_train(train_x, train_y)
            losses.append(loss)
            accs.append(acc)

        # 에폭이 돌면서 학습이 수행되고, 에폭값이 report로 설정한 수에 해당될때 마다 중간평가 수행
        if report > 0 and (epoch+1) % report == 0:
            acc = run_test(test_x, test_y)
            print("Epoch {}: TRAIN-LOSS = {:5.3f}, accuracy = {:5.3f} / Test={:5.3f}".\
                format(epoch+1, np.mean(losses), np.mean(accs), acc))

    final_acc = run_test(test_x, test_y)
    print('\nFinal Test: accuracy = {:5.3f}'.\
        format(final_acc))
